# integrate.ai API Sample Notebook to run tasks with local python.

**NOTE: This notebook only works with the integrate-ai-sdk-embedded package**

This is an example notebook that demonstrates creating taskbuilders and running tasks using local python.

These tasks will run within subprocesses on the current machine.

## Setup
### Set environment variables (or replace inline) with your IAI credentials
Generate and manage this token in the UI, in the Tokens page. 

In [ ]:
from integrate_ai_sdk.api import connect
import os
import json
import pandas as pd

IAI_TOKEN = os.environ.get("IAI_TOKEN")

client = connect(token=IAI_TOKEN)

### Setup Data

You can download sample data from the integrate.ai sample bucket:

For HFL and EDA: [https://s3.ca-central-1.amazonaws.com/public.s3.integrate.ai/integrate_ai_examples/synthetic.zip](https://s3.ca-central-1.amazonaws.com/public.s3.integrate.ai/integrate_ai_examples/synthetic.zip)

For PRL and VFL: [https://s3.ca-central-1.amazonaws.com/public.s3.integrate.ai/integrate_ai_examples/vfl.zip](https://s3.ca-central-1.amazonaws.com/public.s3.integrate.ai/integrate_ai_examples/synthetic.zip)

The following cell will download the data and place it the `/tmp/local_base_dir/` directory outlined below.

In [ ]:
%%bash

mkdir -p /tmp/iai-test-data/
cd /tmp/iai-test-data/
wget -N -q https://s3.ca-central-1.amazonaws.com/public.s3.integrate.ai/integrate_ai_examples/synthetic.zip
wget -N -q https://s3.ca-central-1.amazonaws.com/public.s3.integrate.ai/integrate_ai_examples/vfl.zip
unzip -o synthetic.zip
unzip -o vfl.zip
find ./synthetic
find ./vfl

In [ ]:
local_base_dir = f'/tmp/iai-test-data/'

# Example datapaths. Make sure that the data you want to work with exists in the base_aws_bucket for your task runner.
# HFL datapaths
local_train_path1 = f'{local_base_dir}/synthetic/train_silo0.parquet'
local_test_path1 = f'{local_base_dir}/synthetic/test.parquet'
local_train_path2 = f'{local_base_dir}/synthetic/train_silo1.parquet'
local_test_path2 = f'{local_base_dir}/synthetic/test.parquet'

#EDA/PRL/VFL datapaths
local_active_train_path = f'{local_base_dir}/vfl/active_train.parquet'
local_active_test_path = f'{local_base_dir}/vfl/active_test.parquet'
local_passive_train_path = f'{local_base_dir}/vfl/passive_train.parquet'
local_passive_test_path = f'{local_base_dir}/vfl/passive_test.parquet'

#Where to store the trained model
local_storage_path = f'{local_base_dir}/model/'

#Where to store VFL predictions - must be full path and file name
local_vfl_predict_storage_path = f'{local_base_dir}/vfl_predict/'

local_base_dir

### Set up the taskbuilder 


In [ ]:
import integrate_ai_sdk
from integrate_ai_sdk.taskgroup.taskbuilder.integrate_ai import IntegrateAiTaskBuilder
from integrate_ai_sdk.taskgroup.taskbuilder.local import local_python
from integrate_ai_sdk.taskgroup.base import SessionTaskGroup
from integrate_ai_sdk.taskgroup.taskbuilder import taskrunner_context

iai_tb_local = local_python(client=client)

# Task 1: Perform EDA in Individual Mode

This example task demonstrates how to run an exploratory data analysis (EDA) session in Individual mode.

In [ ]:
# Specify the dataset configuration

dataset_config = {"dataset_one": [], "dataset_two": []}

In [ ]:
# Create and start the EDA session 

eda_session = client.create_eda_session(
    name="Testing notebook - EDA",
    description="I am testing EDA session creation with a task runner through a notebook",
    data_config=dataset_config,
    startup_mode="external"
).start()

eda_session.id  #Prints the EDA session ID for reference

In [ ]:
# Create a task group with one task for the server, and one for each of the clients joining the session

eda_task_group_context = (
        SessionTaskGroup(eda_session) \
        .add_task(iai_tb_local.fls(storage_path=local_storage_path))\
        .add_task(iai_tb_local.eda(dataset_name="dataset_one", dataset_path=local_train_path1))\
        .add_task(iai_tb_local.eda(dataset_name="dataset_two", dataset_path=local_train_path2))\
        .start()
    )

In [ ]:
# Check the task group status

for i in eda_task_group_context.contexts.values():
    print(json.dumps(i.status(), indent=4))

eda_task_group_context.monitor_task_logs()

In [ ]:
# Wait for the tasks to complete (success = True)

eda_task_group_context.wait(60*5, 2)

### Task 1: EDA session complete

In [ ]:
# Session complete, retrieve the results 

#eda_session = client.session('dc214c54eb')

results = eda_session.results()["dataset_one", "dataset_two"]
results

In [ ]:
results.mean().shape

In [ ]:
dataset_one = eda_session.results()["dataset_one"]
dataset_one_count = dataset_one["x0"].count()
dataset_one["x0"].mean()

# Task 2: Create an HFL FFNet Training Session

The documentation for [creating a session](https://documentation.integrateai.net/#tutorial-ffnet-model-training-with-a-sample-local-dataset-iai_ffnet) gives additional context into the parameters that are used during training session creation.<br />
For this session we are going to be using two training clients and two rounds. 

You can find the model config and data schema details in the [integrate.ai end user tutorial](https://documentation.integrateai.net/#understanding-models).

In [ ]:
# Specify the model and data configurations

model_config = {
    "experiment_name": "test_synthetic_tabular",
    "experiment_description": "test_synthetic_tabular",
    "strategy": {"name": "FedAvg", "params": {}},
    "model": {"params": {"input_size": 15, "hidden_layer_sizes": [6, 6, 6], "output_size": 2}},
    "balance_train_datasets": False,
    "ml_task": {
        "type": "classification",
        "params": {
            "loss_weights": None,
        },
    },
    "optimizer": {"name": "SGD", "params": {"learning_rate": 0.2, "momentum": 0.0}},
    "differential_privacy_params": {"epsilon": 4, "max_grad_norm": 7},
    "save_best_model": {
        "metric": "loss",  # to disable this and save model from the last round, set to None
        "mode": "min",
    },
    "seed": 23,  # for reproducibility
}

data_schema = {
    "predictors": ["x0", "x1", "x2", "x3", "x4", "x5", "x6", "x7", "x8", "x9", "x10", "x11", "x12", "x13", "x14"],
    "target": "y",
}

In [ ]:
# Create and start the training session

training_session = client.create_fl_session(
    name="Testing notebook",
    description="I am testing session creation with a task runner through a notebook",
    min_num_clients=2,
    num_rounds=2,
    package_name="iai_ffnet",
    model_config=model_config,
    data_config=data_schema,
    startup_mode="external"
).start()

training_session.id # Prints the training session ID for reference

In [ ]:
# Create a task group with one task for the server, and one for each of the clients joining the session

task_group = (
    SessionTaskGroup(training_session)
    .add_task(iai_tb_local.fls(storage_path=local_storage_path))\
    .add_task(iai_tb_local.hfl(train_path=local_train_path1, test_path=local_test_path1, use_gpu=False))\
    .add_task(iai_tb_local.hfl(train_path=local_train_path2, test_path=local_test_path2))
)

In [ ]:
task_group_context = task_group.start()

In [ ]:
# Monitor the submitted tasks

for i in task_group_context.contexts.values():
    print(json.dumps(i.status(), indent=4))

task_group_context.monitor_task_logs()

In [ ]:
# Wait for the tasks to complete (success = True)

task_group_context.wait(60*5, 2)

### Task 2: HFL Session Complete!
Now you can view the training metrics and start making predictions

In [ ]:
# Retrieve the session metrics

training_session.metrics().as_dict()

In [ ]:
# Plot the session metrics

fig = training_session.metrics().plot()

# Task 3: Create a PRL Session for linking two or more datasets

To create a PRL session, specify a `dataset_config` dictionary indicating the client names and columns to use as identifiers to link the datasets to each other. The number of expected clients will be inferred as the number of items in dataset_config (i.e., two). These client names are referenced for the compute on the PRL session and for any sessions that use the PRL session downstream.

For this session, two clients are going to be providing data. Client 1 and client 2 are naming their clients client_1 and client_2 respectively. Their datasets will be linked by the "id" column in any provided datasets.

In [ ]:
# Specify PRL dataset configuration 

prl_data_config = {
    "clients": {
        "active_client": {"id_columns": ["id"]},
        "passive_client": {"id_columns": ["id"]},
    }
}

In [ ]:
# Create and start PRL session

prl_session = client.create_prl_session(
    name="Testing notebook - PRL",
    description="I am testing PRL session creation with a task runner through a notebook",
    data_config=prl_data_config,
    startup_mode="external"
).start()

prl_session.id #Prints the session ID for reference

In [ ]:
# Create a task group with one task for the server, and one for each of the clients joining the session

task_group = (SessionTaskGroup(prl_session)\
.add_task(iai_tb_local.fls(storage_path=local_storage_path))\
.add_task(iai_tb_local.prl(train_path=local_active_train_path, test_path=local_active_test_path, client_name="active_client"))\
.add_task(iai_tb_local.prl(train_path=local_passive_train_path, test_path=local_passive_test_path, client_name="passive_client"))
)

task_group_context = task_group.start()

In [ ]:
#Check the status of the task group

for i in task_group_context.contexts.values():
    print(json.dumps(i.status(), indent=4))
    
task_group_context.monitor_task_logs()

In [ ]:
# Wait for the tasks to complete (success = True)

task_group_context.wait(60*5, 2)

### Task 3: PRL Session Complete!
Now you can view the overlap stats for the datasets.

In [ ]:
# View PRL session metrics

metrics = prl_session.metrics().as_dict()
metrics

## Task 3a: Create a VFL Training Session using the PRL session from Task 3

To create a VFL train session, specify the `prl_session_id` indicating the session you just ran to link the datasets together. The `vfl_mode` needs to be set to `train`.

For more information about vertical federated learning with a SplitNN model strategy, see [VFL SplitNN Model Trianing](https://documentation.integrateai.net/#vfl-splitnn-model-training). 

In [ ]:
# Specify the model and data configurations

model_config = {
    "strategy": {"name": "SplitNN", "params": {}},
    "model": {
        "feature_models": {
            "passive_client": {"params": {"input_size": 7, "hidden_layer_sizes": [6], "output_size": 5}},
            "active_client": {"params": {"input_size": 8, "hidden_layer_sizes": [6], "output_size": 5}},
        },
        "label_model": {"params": {"hidden_layer_sizes": [5], "output_size": 2}},
    },
    "ml_task": {
        "type": "classification",
        "params": {
            "loss_weights": None,
        },
    },
    "optimizer": {"name": "SGD", "params": {"learning_rate": 0.2, "momentum": 0.0}},
    "seed": 23,  # for reproducibility
}

data_config = {
        "passive_client": {
            "label_client": False,
            "predictors": ["x1", "x3", "x5", "x7", "x9", "x11", "x13"],
            "target": None,
        },
        "active_client": {
            "label_client": True,
            "predictors": ["x0", "x2", "x4", "x6", "x8", "x10", "x12", "x14"],
            "target": "y",
        },
    }

In [ ]:
# Create and start a VFL training session

vfl_train_session = client.create_vfl_session(
    name="Testing notebook - VFL Train",
    description="I am testing VFL Train session creation with a task runner through a notebook",
    prl_session_id=prl_session.id,
    vfl_mode='train',
    min_num_clients=2,
    num_rounds=2,
    package_name="iai_ffnet",
    data_config=data_config,
    model_config=model_config,
    startup_mode="external"
).start()

vfl_train_session.id    #Prints the session ID for reference

In [ ]:
# Create and start a task group with one task for the server, and one for each of the clients joining the session
import pathlib

local_session_storage_path = f"{local_storage_path}/vfl/{vfl_train_session.id}"
pathlib.Path(local_session_storage_path).mkdir(parents=True, exist_ok=True)

vfl_task_group_context = (SessionTaskGroup(vfl_train_session)\
    .add_task(iai_tb_local.fls(storage_path=local_session_storage_path))\
    .add_task(iai_tb_local.vfl_train(train_path=local_active_train_path, 
                                    test_path=local_active_test_path, 
                                    batch_size=1024,  
                                    client_name="active_client", 
                                    storage_path = local_session_storage_path))\
    .add_task(iai_tb_local.vfl_train(train_path=local_passive_train_path, 
                                    test_path=local_passive_test_path, 
                                    batch_size=1024, 
                                    client_name="passive_client", 
                                    storage_path = local_session_storage_path))\
    .start())


In [ ]:
# Check the status of the tasks

for i in vfl_task_group_context.contexts.values():
    print(json.dumps(i.status(), indent=4))

vfl_task_group_context.monitor_task_logs()

In [ ]:
# Wait for the tasks to complete (success = True)

vfl_task_group_context.wait(60*5, 2)

### Task 3a: VFL Session Complete!
Now you can view the VFL training metrics and start making predictions

In [ ]:
metrics = vfl_train_session.metrics().as_dict()
metrics

In [ ]:
fig = vfl_train_session.metrics().plot()

## Task 3b: Make a Prediction on the trained VFL model

To create a VFL predict session, specify the `prl_session_id` indicating the session you ran to link the datasets together. You also need the `training_id` of the VFL train session that was run using the same `prl_session_id`. 

The `vfl_mode` must be set to `predict`.

In [ ]:
# Create and start a VFL predict session

vfl_predict_session = client.create_vfl_session(
    name="Testing notebook - VFL Predict",
    description="I am testing VFL Predict session creation with an AWS task runner through a notebook",
    prl_session_id=prl_session.id,
    training_session_id=vfl_train_session.id,
    vfl_mode="predict",
    data_config=data_config,
    startup_mode="external"
).start()

vfl_predict_session.id  # Prints the session ID for reference

In [ ]:
# Create and start a task group with one task for the server, and one for each of the clients joining the session
import pathlib

pathlib.Path(local_vfl_predict_storage_path + '/active').mkdir(parents=True, exist_ok=True)
pathlib.Path(local_vfl_predict_storage_path + '/passive').mkdir(parents=True, exist_ok=True)

local_vfl_predict_active_storage_path = local_vfl_predict_storage_path + f'/active/{vfl_predict_session.id}.csv'
local_vfl_predict_passive_storage_path = local_vfl_predict_storage_path + f'/passive/{vfl_predict_session.id}.csv'

vfl_predict_task_group_context = (SessionTaskGroup(vfl_predict_session)\
.add_task(iai_tb_local.fls(storage_path=local_session_storage_path))\
.add_task(iai_tb_local.vfl_predict(
        client_name="active_client", 
        dataset_path=local_active_test_path, 
        raw_output=True,
        batch_size=1024, 
        storage_path = local_vfl_predict_active_storage_path))\
.add_task(iai_tb_local.vfl_predict(
        client_name="passive_client",
        dataset_path=local_passive_test_path,
        batch_size=1024,
        raw_output=True,
        storage_path = local_vfl_predict_passive_storage_path))\
.start())

In [ ]:
# Check the status of the tasks

for i in vfl_predict_task_group_context.contexts.values():
    print(json.dumps(i.status(), indent=4))

vfl_predict_task_group_context.monitor_task_logs()

In [ ]:
# Wait for the tasks to complete (success = True)

vfl_predict_task_group_context.wait(60*5, 2)

### Task 3b: VFL Predict Session Complete!

Now you can view the VFL predictions and evaluate the performance as needed

In [ ]:
# Retrieve the metrics

metrics = vfl_predict_session.metrics().as_dict()
metrics

In [ ]:
df_pred = pd.read_csv(local_vfl_predict_active_storage_path)

df_pred.head()

# Task 4: Create a linear inference session

In [ ]:
# Specify the model and data configurations

model_config_logit = {
    "strategy": {"name": "LogitRegInference", "params": {}},
    "seed": 23,  # for reproducibility
}

data_config_logit = {
    "target": "y",
    "shared_predictors": ["x1", "x2"],
    "chunked_predictors": ["x0", "x3", "x10", "x11"]
}

In [ ]:
# Create and start a linear inference session 

training_session_logit = client.create_fl_session(
    name="Testing linear inference session",
    description="I am testing linear inference session creation using a task runner through a notebook",
    min_num_clients=2,
    num_rounds=5,
    package_name="iai_linear_inference",
    model_config=model_config_logit,
    data_config=data_config_logit,
    startup_mode="external"
).start()

training_session_logit.id

In [ ]:
#Create a task group

task_group_context = (
    SessionTaskGroup(training_session_logit)
    .add_task(iai_tb_local.fls(storage_path=local_storage_path))\
    .add_task(iai_tb_local.hfl(train_path=local_train_path1, test_path=local_test_path1))
    .add_task(iai_tb_local.hfl(train_path=local_train_path2, test_path=local_test_path2)).start()
)


In [ ]:
for i in task_group_context.contexts.values():
    print(json.dumps(i.status(), indent=4))
task_group_context.monitor_task_logs()

In [ ]:
task_group_context.wait(60*5, 2)

## Task 4 - Session Complete!
Now we can view the training metrics and model details such as the model coefficients and p-values. Note that since there are a bundle of models being trained, the metrics below are the average values of all the models.

In [ ]:
training_session_logit.metrics().as_dict()
training_session_logit.metrics().plot()

### Trained models are accessible from the completed session

The `LinearInferenceModel` object can be retrieved using the model's `as_pytorch` method. And the relevant information such as p-values can be accessed directly from the model object.


In [ ]:
model_logit = training_session_logit.model().as_pytorch()
pv = model_logit.p_values()
pv

The `.summary` method fetches the coefficient, standard error and p-value of the model corresponding to the specified predictor.

In [ ]:
summary_x0 = model_logit.summary("x0")
summary_x0


It is also possible to make predictions with the resulting bundle of models, when the data is loaded by the `ChunkedTabularDataset` from the `iai_linear_inference` package. For an example of this, see the `integrateai_linear_inference.ipynb` notebook in the [sample repo](https://github.com/integrateai/integrateai-samples/tree/main/sample_notebook).